Regular `pip` is slow, but can be a bit more robust than `uv`

`pandas` may take a while to install. If you get "Runtime Disconnected" just resume and re-run the previous command.

Though sometimes "Runtime Disconnected" is a false message and you're still actually connected! (??)

## IGNORE ANY ERRORS AND JUST KEEP GOING.

You may need to restart at least once though (see below)

In [ ]:
!pip install sentencepiece protobuf numpy pandas PyWavelets torch==2.8.0 torchaudio torchmetrics transformers | grep -v 'already satisfied'

In [ ]:
!pip install alias-free-torch==0.0.6 descript-audio-codec==1.0.0 einops einops-exts ema-pytorch==0.2.3 'gradio>=5.20.0' huggingface_hub 'importlib-resources>=5.12.0' k-diffusion==0.1.1 local-attention==1.8.6 prefigure==0.0.9 pytorch_lightning==2.1.0 safetensors sentencepiece v-diffusion-pytorch==0.0.2 vector-quantize-pytorch==1.14.41 'wandb>=0.15.4' 'webdataset>=0.2.100' torchcodec matplotlib librosa julius | grep -v 'already satisfied'

# ^ IGNORE ERRORS.  Restart if prompted. Keep going.

In [ ]:
!pip install auraloss==0.4.0 encodec==0.1.1 laion-clap==1.1.4 --no-deps  | grep -v 'already satisfied'

In [ ]:
!pip install stable-audio-tools --no-deps  | grep -v 'already satisfied'

In [ ]:
!pip install -U  protobuf

STOP.  Restart Runtime, then refresh browser page

In [ ]:
import stable_audio_tools
print("Success!")

In [ ]:
#import numpy as np
import torch
import torchaudio
from einops import rearrange
from stable_audio_tools import get_pretrained_model
from stable_audio_tools.inference.generation import generate_diffusion_cond
from IPython.display import Audio


device = "cuda" if torch.cuda.is_available() else "mps"if torch.backends.mps.is_available() else "cpu"
print("device = ",device)

In [ ]:
from huggingface_hub import login

token = "######__REPLACE_WITH_YOUR_HUGGINGFACE_MODEL_API_TOKEN__######"
login(token=token)

In [ ]:
# Download model
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-small")
print("model_config=",model_config)
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]
model = model.to(device)

## ^^^ IGNORE ANY ERRORS. Keep going!

In [ ]:
# Set up text and timing conditioning
conditioning = [{
    "prompt": "60 BPM jazz saxophone solo",
    "seconds_total": 11
}]

In [ ]:
# Generate stereo audio
output = generate_diffusion_cond(
    model,
    steps=8,
    cfg_scale=1.0,
    conditioning=conditioning,
    sample_size=sample_size,
    sampler_type="pingpong",
    device=device,
    callback=None,
)
# Rearrange audio batch to a single sequence
output = rearrange(output, "b d n -> d (b n)")

In [ ]:
output = output.to(torch.float32).div(torch.max(torch.abs(output))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()
torchaudio.save("output.wav", output, sample_rate)

In [ ]:
Audio("output.wav")